In [1]:
from tqdm import tqdm
import pandas as pd
import json
import os
import ast
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

import wandb
api = wandb.Api()

In [3]:
setting_columns = [
    # Overall
    "tag",
    # "seed",
    "_timestamp",

    # Interventions
    "interventions", "edit", "unlearn", "compression", "model_name",

    # Editing
    # "edit_set", 
    "edit_dataset", "number_of_edits",

    # Unlearning
    "rmu_layer_id",

    # Compression
    "wbits", "compression_dataset", "sparsity_ratio",
]
evaluation_columns = [
    "qa_question_count_limit",  # An artifical max number of questions to ask during evaluation. Should be none when not debugging.
    "mmlu accuracy",            # The accuracy of the model on the MMLU dataset. This measures overall model utility. Llama-3 should be ~62%
    "wmdp_bio accuracy",        # The accuracy of the model on the WMDP bio split. This is the unlearning target. Should be ~25% when RMU is applied.
    "wmdp_cyber accuracy",      # The accuracy of the model on the WMDP cyber split. This is the unlearning target. Should be ~25% when RMU is applied.
    "PPL",                      # TODO:
    "PPL edits",                # Perplexity for the edits. Should be low when editing is applied.
    "PPl QA",                   # Perplexity for the QA. Should be low when QA is applied.
    "Generalization",           # TODO: 
    "FLOPs",                    # TODO: 
    "Success recall",           # TODO:
    "Generalization recall",    # TODO:
    "Locality",                 # TODO:
    "Average bits",             # TODO:
    "Rewrite accuracy",         # TODO:
    "PPl edits unmasked",       # TODO:
    "Local recall",             # TODO:
    "Latency",                  # TODO:
]
relevant_columns = setting_columns + evaluation_columns

In [4]:
# Replace 'username/project_name' with your specific project path
# Composable_Interventions
project_paths = [
    'dri-ice/Composable_Interventions',
    # 'dri-ice/AK_Tests'
]

filter_dict = { 
    "state": "finished",
    # "created_at": {"$gte": "2024-05-20"}
}
data_frames = []
for project_path in project_paths:
    runs = api.runs(project_path, filters=filter_dict)
    
    # Iterate over eachrun and capture the config and summary metrics
    for run in tqdm(runs, desc=project_path):
        try:
            run_start_datetime = datetime.fromtimestamp(run.summary_metrics["_timestamp"])
            start_cutoff = datetime.strptime("2024-05-18 00:00:00", "%Y-%m-%d %H:%M:%S")
            end_cutoff = datetime.strptime("2024-05-23 00:00:00", "%Y-%m-%d %H:%M:%S")
            if run_start_datetime < start_cutoff or run_start_datetime > end_cutoff:
                continue

            skip_tags = ["test", "hparam_search"]
            should_skip = False
            for tag in skip_tags:
                if tag in run.config["tag"].lower():
                    should_skip = True
            
            if should_skip:
                continue

            config_frame = pd.DataFrame([run.config])
            summary_frame = pd.DataFrame([run.summary_metrics])
            combined_frame = pd.concat([config_frame, summary_frame], axis=1)
            data_frames.append(combined_frame)
        except Exception as e:
            print(f"Error processing run {run.id}: {e}")

# Sort by 'tag' and '_timestamp' in descending order to have the most recent run first
all_runs_df = pd.concat(data_frames, ignore_index=True)[relevant_columns]
all_runs_df["interventions"] = all_runs_df["interventions"].astype(str)

# WARNING: WHAT DOES EDIT SET 50 MEAN COMPARED TO EDIT SET 1?
# all_runs_df = all_runs_df[all_runs_df["edit_set"] == 50]
# all_runs_df_sorted = all_runs_df.sort_values(by=['tag', '_timestamp'], ascending=[True, False])
all_runs_df["date"] = pd.to_datetime(all_runs_df["_timestamp"], unit='s')
all_runs_df_sorted = all_runs_df.sort_values(by=['_timestamp'], ascending=[False])

dri-ice/Composable_Interventions: 100%|██████████| 303/303 [00:02<00:00, 148.50it/s]

Error processing run n0iel6ok: '_timestamp'
Error processing run xr5mede5: '_timestamp'
Error processing run 27f8pxs0: '_timestamp'


In [5]:
all_runs_df_sorted

,tag,_timestamp,interventions,edit,unlearn,compression,model_name,edit_dataset,number_of_edits,rmu_layer_id,...,FLOPs,Success recall,Generalization recall,Locality,Average bits,Rewrite accuracy,PPl edits unmasked,Local recall,Latency,date
0,rmu-none,1.716351e+09,['unlearn'],none,rmu,none,meta-llama/Meta-Llama-3-8B,zsre,50,3.0,...,1.92 TFLOPS,0.008889,0.020667,0.027140,16.000000,0.008889,5.251807e+02,0.027449,96.211963,2024-05-22 04:14:27.846383810
11,AWQ8bit-to-lora,1.716351e+09,"['compress', 'edit']",lora,none,awq,meta-llama/Meta-Llama-3-8B,zsre,50,-1.0,...,-1,1.000000,0.630000,0.052772,8.250000,1.000000,1.393250e+04,0.052387,515.406013,2024-05-22 04:10:40.129939795
13,AWQ2bit-to-lora,1.716351e+09,"['compress', 'edit']",lora,none,awq,meta-llama/Meta-Llama-3-8B,zsre,50,-1.0,...,-1,0.070556,0.040000,0.000000,2.250000,0.089636,7.574332e+04,0.000000,519.011764,2024-05-22 04:05:45.239004850
12,AWQ4bit-to-lora,1.716351e+09,"['compress', 'edit']",lora,none,awq,meta-llama/Meta-Llama-3-8B,zsre,50,-1.0,...,-1,1.000000,0.750000,0.085162,4.250000,1.000000,1.342348e+04,0.085194,519.676339,2024-05-22 04:05:43.669849396
2,ft-rmu,1.716351e+09,"['edit', 'unlearn']",ft,rmu,none,meta-llama/Meta-Llama-3-8B,zsre,50,5.0,...,1.92 TFLOPS,1.000000,0.784444,0.114612,15.999968,1.000000,6.431615e+02,0.114261,101.909465,2024-05-22 04:04:48.395482302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,Compress_AWQ2bit,1.716224e+09,['compress'],none,none,awq,meta-llama/Meta-Llama-3-8B,zsre,50,NaN,...,-1,0.000000,0.000000,0.000000,2.250000,0.000000,1.020036e+06,0.000000,87.598504,2024-05-20 16:59:38.218446016
281,Compress_AWQ8bit,1.716224e+09,['compress'],none,none,awq,meta-llama/Meta-Llama-3-8B,zsre,50,NaN,...,-1,0.006667,0.010000,0.030760,8.250000,0.006667,4.443523e+02,0.030682,88.839530,2024-05-20 16:59:32.776091576
280,Compress_AWQ4bit,1.716224e+09,['compress'],none,none,awq,meta-llama/Meta-Llama-3-8B,zsre,50,NaN,...,-1,0.012381,0.000000,0.030575,4.250000,0.012821,4.668763e+02,0.029872,87.954436,2024-05-20 16:59:29.555692434
278,Compress_AWQ2bit,1.716224e+09,['compress'],none,none,awq,meta-llama/Meta-Llama-3-8B,zsre,50,NaN,...,-1,0.000000,0.000000,0.000000,2.250000,0.000000,1.020036e+06,0.000000,87.735735,2024-05-20 16:58:04.464708805


In [6]:
# Drop duplicates, keeping only the first occurrence (which is the most recent due to sorting)
# all_runs_df_deduplicated = all_runs_df_sorted.drop_duplicates(subset=[col for col in setting_columns if col not in ["_timestamp", "tag", "date"]], keep="first")
all_runs_df_deduplicated = all_runs_df_sorted.drop_duplicates(subset="tag", keep="first")
# all_runs_df_deduplicated = all_runs_df_sorted
all_runs_df_deduplicated["interventions"] = all_runs_df_deduplicated["interventions"].apply(lambda x : ast.literal_eval(x))

rename_dict = {
    "meta-llama/Meta-Llama-3-8B" : "Llama-3 (8b)",
    "ft" : "Fine-tune",
    "memit" : "MEMIT",
    "lora" : "Lora",
    "wanda" : "Wanda",
    "sparsegpt" : "SparseGPT",
    "gptq" : "GPTQ",
    "awq" : "AWQ",
    "rmu" : "RMU",
}
metrics = all_runs_df_deduplicated
metrics["model_name"] = metrics["model_name"].apply(lambda x : rename_dict.get(x, None))
metrics["edit"] = metrics["edit"].apply(lambda x : rename_dict.get(x, None))
metrics["compression"] = metrics["compression"].apply(lambda x : rename_dict.get(x, None))
metrics["unlearn"] = metrics["unlearn"].apply(lambda x : rename_dict.get(x, None))
all_runs_df_deduplicated = metrics
all_runs_df_deduplicated.value_counts("tag")

/tmp/ipykernel_139302/4290825372.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_runs_df_deduplicated["interventions"] = all_runs_df_deduplicated["interventions"].apply(lambda x : ast.literal_eval(x))
/tmp/ipykernel_139302/4290825372.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics["model_name"] = metrics["model_name"].apply(lambda x : rename_dict.get(x, None))
/tmp/ipykernel_139302/4290825372.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

tag
AWQ2bit-to-ft              1
lora-to-SparseGPT0.25%     1
lora-to-AWQ3bit            1
lora-to-AWQ4bit            1
lora-to-AWQ5bit            1
                          ..
SparseGPT0.55%-to-memit    1
SparseGPT0.65%-to-ft       1
SparseGPT0.65%-to-lora     1
SparseGPT0.65%-to-memit    1
wanda0.65\%-rmu            1
Name: count, Length: 186, dtype: int64

In [7]:
all_runs_df_deduplicated[all_runs_df_deduplicated["compression"] == "SparseGPT"]

,tag,_timestamp,interventions,edit,unlearn,compression,model_name,edit_dataset,number_of_edits,rmu_layer_id,...,FLOPs,Success recall,Generalization recall,Locality,Average bits,Rewrite accuracy,PPl edits unmasked,Local recall,Latency,date
52,SparseGPT0.65%-to-memit,1.716348e+09,"[compress, edit]",MEMIT,None,SparseGPT,Llama-3 (8b),zsre,50,-1.0,...,759.86 GFLOPS,0.116222,0.057889,0.010900,6.249982,0.116667,1358.275513,0.010438,-1.0,2024-05-22 03:17:58.787498713
56,SparseGPT0.25%-to-memit,1.716348e+09,"[compress, edit]",MEMIT,None,SparseGPT,Llama-3 (8b),zsre,50,-1.0,...,1.47 TFLOPS,0.820000,0.800000,0.041359,12.249977,0.819231,451.862762,0.042053,-1.0,2024-05-22 03:15:44.578335047
54,SparseGPT0.45%-to-memit,1.716348e+09,"[compress, edit]",MEMIT,None,SparseGPT,Llama-3 (8b),zsre,50,-1.0,...,1.12 TFLOPS,0.649556,0.435111,0.023404,9.249988,0.649556,458.593323,0.023942,-1.0,2024-05-22 03:15:22.386393309
76,SparseGPT0.45%-to-lora,1.716347e+09,"[compress, edit]",Lora,None,SparseGPT,Llama-3 (8b),zsre,50,-1.0,...,982.73 GFLOPS,0.746111,0.214667,0.025034,9.249988,0.746765,1741.787231,0.024861,-1.0,2024-05-22 02:55:08.137873888
75,SparseGPT0.65%-to-lora,1.716346e+09,"[compress, edit]",Lora,None,SparseGPT,Llama-3 (8b),zsre,50,-1.0,...,625.37 GFLOPS,0.441556,0.181889,0.020420,6.249982,0.443071,1556.614868,0.020079,-1.0,2024-05-22 02:54:03.460087299
73,SparseGPT0.45%-to-ft,1.716346e+09,"[compress, edit]",Fine-tune,None,SparseGPT,Llama-3 (8b),zsre,50,-1.0,...,1.12 TFLOPS,0.929667,0.644667,0.048091,9.249988,0.929667,561.806335,0.047011,-1.0,2024-05-22 02:51:42.515434980
72,SparseGPT0.65%-to-ft,1.716346e+09,"[compress, edit]",Fine-tune,None,SparseGPT,Llama-3 (8b),zsre,50,-1.0,...,759.86 GFLOPS,0.972222,0.543889,0.010484,6.249982,0.974762,1071.286987,0.010011,-1.0,2024-05-22 02:51:32.372437716
74,SparseGPT0.25%-to-ft,1.716346e+09,"[compress, edit]",Fine-tune,None,SparseGPT,Llama-3 (8b),zsre,50,-1.0,...,1.47 TFLOPS,0.995000,0.663333,0.080201,12.249977,0.995000,518.634399,0.080455,-1.0,2024-05-22 02:50:44.011991501
77,SparseGPT0.25%-to-lora,1.716346e+09,"[compress, edit]",Lora,None,SparseGPT,Llama-3 (8b),zsre,50,-1.0,...,1.34 TFLOPS,0.898000,0.517667,0.032262,12.249977,0.900000,2683.486572,0.032788,-1.0,2024-05-22 02:50:15.668058157
66,memit-to-SparseGPT0.25%,1.716345e+09,"[edit, compress]",MEMIT,None,SparseGPT,Llama-3 (8b),zsre,50,NaN,...,1.47 TFLOPS,0.776111,0.705000,0.032233,12.249977,0.776111,472.074982,0.032887,-1.0,2024-05-22 02:30:45.327311277


In [8]:
# Math for determining number of interventions
awq_settings = 5
gptq_settings = 4 # only support quantize to [2, 3, 4, 8] bits.
wanda_count = 6
sparsegpt_count = 6
editor_settings = 3
composition_factor = 2

editor_count = composition_factor * (awq_settings + gptq_settings + wanda_count + sparsegpt_count + 1) * editor_settings
print(editor_count // 2)

rmu_count = composition_factor * (awq_settings + gptq_settings + wanda_count + sparsegpt_count + editor_settings)
print(rmu_count)

66
48


In [9]:
data = all_runs_df_deduplicated

categories = {
    "No Intervention": data[data["interventions"].apply(lambda x: x == [])].copy(),
    "Editing": data[data["interventions"].apply(lambda x: x == ["edit"])].copy(),
    "Compression": data[data["interventions"].apply(lambda x: x == ["compress"])].copy(),
    "Edit to Compression": data[data["interventions"].apply(lambda x: x == ["edit", "compress"])].copy(),
    "Compression to Edit": data[data["interventions"].apply(lambda x: x == ["compress", "edit"])].copy(),
    "Unlearn": data[data["interventions"].apply(lambda x: x == ["unlearn"])].copy(),
    "Edit to Unlearn": data[data["interventions"].apply(lambda x: x == ["edit", "unlearn"])].copy(),
    "Unlearn to Edit": data[data["interventions"].apply(lambda x: x == ["unlearn", "edit"])].copy(),
    "Compress to Unlearn": data[data["interventions"].apply(lambda x: x == ["compress", "unlearn"])].copy(),
    "Unlearn to Compress": data[data["interventions"].apply(lambda x: x == ["unlearn", "compress"])].copy()
}

# assert len(categories["No Intervention"]) == 0 # Should be 1

# display(categories["Editing"])
assert len(categories["Editing"]) == 3, f"{len(categories['Editing'])} != 3"

# display(categories["Compression"])
# assert len(categories["Compression"]) == (awq_settings + gptq_settings + wanda_count + sparsegpt_count), f"{len(categories['Compression'])} != {awq_settings + gptq_settings + wanda_count + sparsegpt_count}"

# display(categories["Edit to Compression"])
assert len(categories["Edit to Compression"]) == editor_count // 2, f"{len(categories['Edit to Compression'])} != {editor_count // 2}"

# display(categories["Compression to Edit"])
# Missing Wanda0.25%-to-lora, Wanda0.45%-to-lora, Wanda0.65%-to-lora, 
assert len(categories["Compression to Edit"]) == (editor_count // 2 )- 3, f"{len(categories['Compression to Edit'])} != {editor_count // 2}" # TODO: Fix this by getting the latest results

display(categories["Unlearn"].iloc[0])
assert len(categories["Unlearn"]) == 1, f"{len(categories['Unlearn'])} != 1"

display(categories["Edit to Unlearn"])
assert len(categories["Edit to Unlearn"]) == 0

display(categories["Unlearn to Edit"])
assert len(categories["Unlearn to Edit"]) == 0

display(categories["Compress to Unlearn"])
assert len(categories["Compress to Unlearn"]) == 0

display(categories["Unlearn to Compress"])
assert len(categories["Unlearn to Compress"]) == 0


tag                                             rmu-none
_timestamp                             1716351267.846384
interventions                                  [unlearn]
edit                                                None
unlearn                                              RMU
compression                                         None
model_name                                  Llama-3 (8b)
edit_dataset                                        zsre
number_of_edits                                       50
rmu_layer_id                                         3.0
wbits                                                 16
compression_dataset                                   c4
sparsity_ratio                                       0.0
qa_question_count_limit                             None
mmlu accuracy                                   0.563168
wmdp_bio accuracy                               0.252946
wmdp_cyber accuracy                             0.273276
PPL                            

,tag,_timestamp,interventions,edit,unlearn,compression,model_name,edit_dataset,number_of_edits,rmu_layer_id,...,FLOPs,Success recall,Generalization recall,Locality,Average bits,Rewrite accuracy,PPl edits unmasked,Local recall,Latency,date
2,ft-rmu,1.716351e+09,"[edit, unlearn]",Fine-tune,RMU,None,Llama-3 (8b),zsre,50,5.0,...,1.92 TFLOPS,1.00,0.784444,0.114612,15.999968,1.00,643.161499,0.114261,101.909465,2024-05-22 04:04:48.395482302
1,memit-rmu,1.716351e+09,"[edit, unlearn]",MEMIT,RMU,None,Llama-3 (8b),zsre,50,5.0,...,1.92 TFLOPS,0.99,0.886667,0.047172,16.000000,0.99,518.951538,0.046733,94.809404,2024-05-22 04:03:53.009504080
238,lora-rmu,1.716237e+09,"[edit, unlearn]",Lora,RMU,None,Llama-3 (8b),zsre,50,5.0,...,1.79 TFLOPS,1.00,0.618571,0.057981,16.000000,1.00,25660.986328,0.058719,512.197093,2024-05-20 20:31:09.500632286


AssertionError: 

In [ ]:
def format_flops(value):
    """ Format FLOPs with three significant figures and appropriate suffix. """
    try:
        if isinstance(value, str):
            value = clean_numeric_value(value)
        if abs(value) < 1e6:  # Less than 1 million (below Mega)
            return "{:.3g}k".format(value / 1e3)
        elif abs(value) < 1e9:  # Mega to Giga range
            return "{:.3g}M".format(value / 1e6)
        elif abs(value) < 1e12:  # Giga to Tera range
            return "{:.3g}G".format(value / 1e9)
        else:  # Tera and above
            return "{:.3g}T".format(value / 1e12)
    except Exception as e:
        print(f"Error formatting FLOPs value {value}: {e}")
        return "---"

def escape_latex_special_chars(s):
    """ Escape special characters in LaTeX strings. """
    return str(s).replace('%', '\\%').replace('_', '\\_').replace('&', '\\&').replace('#', '\\#').replace('$', '\\$')

def clean_numeric_value(value):
    """ Convert a string with units to a numeric value. """
    try:
        value = str(value)
        if ' TFLOPS' in value:
            return float(value.replace(' TFLOPS', '')) * 1e12
        if ' GFLOPS' in value:
            return float(value.replace(' GFLOPS', '')) * 1e9
        if ' MFLOPS' in value:
            return float(value.replace(' MFLOPS', '')) * 1e6
        if ' kFLOPS' in value:
            return float(value.replace(' kFLOPS', '')) * 1e3
        return pd.to_numeric(value, errors='coerce')
    except Exception as e:
        print(f"Error cleaning value {value}: {e}")
        return pd.NA

def categorize_and_generate_latex(data):
    # Define categories based on the provided criteria
    categories = {
    "No Intervention": data[data['interventions'].apply(lambda x: x == [])].copy(),
    "Editing": data[data['interventions'].apply(lambda x: x == ['edit'])].copy(),
    "Compression": data[data['interventions'].apply(lambda x: x == ['compress'])].copy(),
    "Edit to Compression": data[data['interventions'].apply(lambda x: x == ['edit', 'compress'])].copy(),
    "Compression to Edit": data[data['interventions'].apply(lambda x: x == ['compress', 'edit'])].copy(),
    "Unlearn": data[data['interventions'].apply(lambda x: x == ['unlearn'])].copy(),
    "Edit to Unlearn": data[data['interventions'].apply(lambda x: x == ['edit', 'unlearn'])].copy(),
    "Unlearn to Edit": data[data['interventions'].apply(lambda x: x == ['unlearn', 'edit'])].copy(),
    "Compress to Unlearn": data[data['interventions'].apply(lambda x: x == ['compress', 'unlearn'])].copy(),
    "Unlearn to Compress": data[data['interventions'].apply(lambda x: x == ['unlearn', 'compress'])].copy()
}
    # Clean numeric columns
    for col in ["FLOPs", "Latency"]:
        if col in data.columns:
            data.loc[:, col] = data[col].apply(clean_numeric_value)
            data.loc[:, col] = pd.to_numeric(data[col], errors='coerce')  # Ensure all values are numeric

    # Column mappings
    column_mappings = {
        "Success": "Rewrite accuracy",
        "Generalization": "Generalization",
        "Locality": "Locality",
        "Avg. Bits": "Average bits",
        "FLOPs": "FLOPs",
        "PPL": "PPL",
        "MMLU": "mmlu accuracy",
        "WMDP Bio": "wmdp_bio accuracy",
        "WMDP Cyber": "wmdp_cyber accuracy"
    }
    latex_columns = ["Success", "Generalization", "Locality", "Avg. Bits", "FLOPs", "PPL", "MMLU", "WMDP Bio", "WMDP Cyber"]

    # Initialize output string
    output_str = ""

    for category, group in categories.items():
        if group.empty:
            continue
        # output_str += f"\\textbf{{{category}}} \\\\ \\midline\n"
        for _, row in group.iterrows():
            # Calculate mean and std for each relevant column within the group
            stats = {}
            for latex_col, csv_col in column_mappings.items():
                if csv_col in row.index:
                    value = row[csv_col]
                    if pd.isna(value):
                        stats[latex_col] = "---"
                    else:
                        # Custom formatting for FLOPs and Latency
                        if latex_col == "FLOPs":
                            mean_str = format_flops(value)
                            stats[latex_col] = escape_latex_special_chars(mean_str)
                        elif latex_col == "Latency":
                            mean_str = f"{value:.3f}s"
                            stats[latex_col] = escape_latex_special_chars(mean_str)
                        else:
                            mean_str = f"{value:.3f}"
                            stats[latex_col] = escape_latex_special_chars(mean_str)
                else:
                    stats[latex_col] = "---"

            # Prepare the LaTeX row for the current group
            latex_row = escape_latex_special_chars(row['tag'])  # Use the tag name directly without escaping
            for column in latex_columns:
                latex_row += " & " + stats.get(column, "---")
            latex_row += " \\\\"

            # Append to output string
            output_str += latex_row + "\n"
        
        output_str += "\\midrule\n"

    return output_str

latex_rows_with_categories = categorize_and_generate_latex(all_runs_df_deduplicated)
print(latex_rows_with_categories)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Set the font family to serif
plt.rcParams['font.family'] = 'serif'

# Assuming 'metrics' DataFrame is already loaded from the CSV

# Define the criteria for filtering
selected_method = 'AWQ'
edit_method = ["FT"]

# Filter based on the criteria
edit_then_compress = categories['Edit to Compression']
compress_then_edit = categories['Compression to Edit']

# Filter based on selected method
edit_then_compress = edit_then_compress[edit_then_compress['compression']==selected_method]
compress_then_edit = compress_then_edit[compress_then_edit['compression']==selected_method]

# Add baselines to dfs
baseline = categories['Editing']
baseline['wbits'] = 16
edit_then_compress = pd.concat([edit_then_compress, baseline], axis=0)
compress_then_edit = pd.concat([compress_then_edit, baseline], axis=0)

# Sort by 'wbits' in ascending order
edit_then_compress = edit_then_compress.sort_values(by='wbits')
compress_then_edit = compress_then_edit.sort_values(by='wbits')

# Define the metrics to plot
metrics_to_plot = ['Rewrite accuracy', 'Generalization', 'mmlu']
x_axis_metric = 'wbits'

# Compute baselines
# edit_then_compress_baselines = {model: edit_then_compress[(edit_then_compress['model_name'] == model) & (edit_then_compress['sparsity_ratio'] == 0)][metrics_to_plot].mean() for model in included_models}
# compress_then_edit_baselines = {model: compress_then_edit[(compress_then_edit['model_name'] == model) & (compress_then_edit['sparsity_ratio'] == 0)][metrics_to_plot].mean() for model in included_models}
# edit_then_compress_baselines = categories['No Intervention']
# compress_then_edit_baselines = categories['No Intervention']

# Define plot parameters
title_fontsize = 20
label_fontsize = 20
legend_fontsize = 18
tick_fontsize = 18
line_width = 3
marker_size = 8

# Create subplots
fig, axes = plt.subplots(1, len(metrics_to_plot), figsize=(15, 5))

# Iterate over each metric and plot
for i, metric in enumerate(metrics_to_plot):
    ax = axes[i]
    
    # Plot the data with scatter and lines
    ax.plot(edit_then_compress['wbits'], edit_then_compress[metric], linestyle='--', marker='o', markerfacecolor='purple', color='purple', label='Edit then compress',
            linewidth=line_width, markersize=marker_size, markeredgewidth=line_width)
    ax.plot(compress_then_edit['wbits'], compress_then_edit[metric], linestyle='-', marker='o', markerfacecolor='none', color='purple', label='Compress then edit',
            linewidth=line_width, markersize=marker_size, markeredgewidth=line_width)
    
    # Fill the area between the lines
    ax.fill_between(edit_then_compress['wbits'], edit_then_compress[metric], compress_then_edit[metric], color='purple', alpha=0.2)
    
    # Integrate baselines into the scatter plots
    for model in included_models:
        baseline_edit = edit_then_compress_baselines[metric]
        baseline_compress = compress_then_edit_baselines[metric]
        
        if x_axis_metric == 'Average bits':
            baseline_x = 16
            ax.set_xlim(2, 16)
        elif x_axis_metric == 'sparsity_ratio':
            baseline_x = 0.0
            ax.set_xlim(0, 1)
        else:
            baseline_x = 0  # Adjust based on your default x-axis range

        # Add baselines to the scatter plots
        # ax.scatter([baseline_x], [baseline_edit], color='purple', marker='o', s=marker_size**2, edgecolor='purple', linewidth=line_width)
        # ax.scatter([baseline_x], [baseline_compress], color='purple', marker='o', s=marker_size**2, edgecolor='purple', linewidth=line_width)
    if x_axis_metric == 'wbits':
        ax.set_xlabel('Bits', fontsize=label_fontsize)
    else:
        ax.set_xlabel(x_axis_metric, fontsize=label_fontsize)
    if metric == 'Rewrite accuracy':
        ax.set_ylabel('Edit success', fontsize=label_fontsize)
    else:
        ax.set_ylabel(metric, fontsize=label_fontsize)
    ax.tick_params(axis='both', which='major', labelsize=tick_fontsize)

# Move the legend to the bottom of the figure
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', fontsize=legend_fontsize, ncol=2)

plt.tight_layout(rect=[0, 0.1, 1, 1])  # Adjust the bottom margin to make space for the legend
plt.show()
plt.savefig('figures/memit-gptq.pdf', format='pdf')
